In [46]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

In [20]:
response = requests.get('https://boardgamegeek.com/browse/boardgame')

In [21]:
response.ok

True

In [22]:
response.status_code

200

In [23]:
print(response.text)


<!DOCTYPE html>
<html ng-app="GeekApp" lang="en-US" ng-cloak>
<head>
	<meta charset='utf-8'>
	<meta id="vp" name="viewport" content="width=device-width, initial-scale=1.0">
			<script>
			window.addEventListener( 'DOMContentLoaded',  function() {
				if (window.innerWidth < 960) {
					var mvp = document.getElementById('vp');
					mvp.setAttribute('content','width=960');
				}
			});
		</script>
		<meta content='yes' name='apple-mobile-web-app-capable'>
	<meta content='IE=edge,chrome=1' http-equiv='X-UA-Compatible'>

			<title>Browse Board Games | BoardGameGeek</title>
	
	
<link rel="apple-touch-icon" 	href="https://cf.geekdo-static.com/icons/touch-icon180.png" />
<link rel="shortcut icon" 		href="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />
<link rel="icon" 					href="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />
<link rel="search" 				href="/game-opensearch.xml" type="application/opensearchdescription+xml" title="BGG Game Search" />
<me

In [65]:
response = requests.get("http://www.boardgamegeek.com/xmlapi/boardgame/174430")

In [66]:
soup = BeautifulSoup(response.text)

In [68]:
soup

<html><body><boardgames termsofuse="https://boardgamegeek.com/xmlapi/termsofuse">
<boardgame objectid="174430">
<yearpublished>2017</yearpublished>
<minplayers>1</minplayers>
<maxplayers>4</maxplayers>
<playingtime>120</playingtime>
<minplaytime>60</minplaytime>
<maxplaytime>120</maxplaytime>
<age>12</age>
<name primary="true" sortindex="1">Gloomhaven</name>
<name sortindex="1">Gloomhaven. Мрачная Гавань</name>
<name sortindex="1">Homályrév</name>
<name sortindex="1">幽港迷城</name>
<name sortindex="1">글룸헤이븐</name>
<description>Gloomhaven  is a game of Euro-inspired tactical combat in a persistent world of shifting motives. Players will take on the role of a wandering adventurer with their own special set of skills and their own reasons for traveling to this dark corner of the world. Players must work together out of necessity to clear out menacing dungeons and forgotten ruins. In the process, they will enhance their abilities with experience and loot, discover new locations to explore and

In [115]:
{
    "name" : soup.find("name").text,
    "year" : soup.find("yearpublished").text,
    "min_players" : soup.find("minplayers").text,
    "max_players" : soup.find("maxplayers").text,
    "min_time" : soup.find("minplaytime").text,
    "max_time" : soup.find("maxplaytime").text,
    "age" : soup.find("age").text,
    "description" : soup.find("description").text,
    "family" : soup.find_all("boardgamefamily"),
    "mechanic" : soup.find_all("boardgamemechanic")
}

{'name': 'Gloomhaven',
 'year': '2017',
 'min_players': '1',
 'max_players': '4',
 'min_time': '60',
 'max_time': '120',
 'age': '12',
 'description': 'Gloomhaven  is a game of Euro-inspired tactical combat in a persistent world of shifting motives. Players will take on the role of a wandering adventurer with their own special set of skills and their own reasons for traveling to this dark corner of the world. Players must work together out of necessity to clear out menacing dungeons and forgotten ruins. In the process, they will enhance their abilities with experience and loot, discover new locations to explore and plunder, and expand an ever-branching story fueled by the decisions they make.<br/><br/>This is a game with a persistent and changing world that is ideally played over many game sessions. After a scenario, players will make decisions on what to do, which will determine how the story continues, kind of like a &ldquo;Choose Your Own Adventure&rdquo; book. Playing through a sce